In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import re

In [2]:
df = pd.read_csv('baseDeDados.csv', sep=';')

/media/user/DADOS1/anaconda/envs/iainverno/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def extend_list(lst):
    try:
        lst[2]
    except:
        lst.append('')
    return lst

def list_to_int(lst):
    new_lst = []
    for item in lst:
        try:
            new_lst.append(int(item))
        except:
            continue
    return new_lst

def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)

    # len(s1) >= len(s2)
    if len(s2) == 0:
        return len(s1)

    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1       
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    
    return previous_row[-1]

In [4]:
tmp = df[['id', 'codigo_componente', 'nome_componente', 'nota_discente_componente']]\
.drop_duplicates(subset=['id', 'nome_componente'])\
.drop(columns='id')

medias = tmp\
.groupby(by=['codigo_componente', 'nome_componente'])\
.mean()\
.rename(columns={
    "nota_discente_componente": 'media'
})

# Copiado de http://inf.ufg.br/sites/default/files/uploads/Resolucao_CEPEC_2017_1561.pdf
novos_requisitos = """
01 Álgebra Linear IME 64 0 64 NC OBR
02 Algoritmos e Estruturas de Dados 1 INF 26 32 32 64 NC OBR
03 Algoritmos e Estruturas de Dados 2 INF 2 64 0 64 NC OBR
04 Análise e Projeto de Algoritmos INF 21, 03 64 0 64 NC OBR
05 Arquitetura de Computadores INF 21 48 16 64 NC OBR
06 Banco de Dados INF 29 48 16 64 NC OBR
07 Cálculo 1A IME 96 0 96 NC OBR
08 Cálculo 2A IME 7 96 0 96 NE OBR
09 Cálculo Numérico IME 18 64 0 64 NE OBR
10 Circuitos Digitais EMC 21 64 0 64 NE OBR
11 Compiladores INF 03, 28 48 16 64 NE OBR
12 Computação e Sociedade INF 32 0 32 NC OBR
13 Computação Gráfica INF 22 32 32 64 NE OBR
14 Computação Paralela INF 39 48 16 64 NE OBR
15 Eletrônica para Computação EMC 20 96 0 96 NE OBR
16 Engenharia de Requisitos INF 16 48 64 NE OBR
17 Engenharia de Software INF 64 0 64 NC OBR
18 Equações Diferenciais Ordinárias IME 01, 08 64 0 64 NE OBR
19 Física I IF 7 64 0 64 NE OBR
20 Física III IF 19 64 0 64 NE OBR
21 Fundamentos de Matemática para Computação INF 64 0 64 NC OBR
22 Geometria Analítica IME 64 0 64 NE OBR
23 Inteligência Artificial INF 02, 29, 33 64 0 64 NE OBR
24 Interação Humano-Computador INF 02, 33 32 32 64 NC OBR
25 Introdução à Computabilidade e à Complexidade Computacional INF 04, 28 64 0 64 NE OBR
26 Introdução à Programação INF 48 80 128 NC OBR
27 Linguagens e Paradigmas de Programação INF 26 32 32 64 NC OBR
28 Linguagens Formais e Autômatos INF 21 64 0 64 NE OBR
29 Lógica Matemática INF 21 64 0 64 NC OBR
30 Matemática Discreta INF 21 64 0 64 NE OBR
31 Metodologia de Pesquisa em Computação INF 32 0 32 NE OBR
32 Pesquisa Operacional INF 1 64 0 64 NE OBR
33 Probabilidade e Estatística A IME 7 64 0 64 NC OBR
34 Programação Orientada a Objetos INF 26 32 32 64 NC OBR
35 Projeto de Software INF 17, 34 64 0 64 NC OBR
36 Projeto Final de Curso 1 INF 0 64 64 NE OBR
37 Projeto Final de Curso 2 INF 36 0 64 64 NE OBR
38 Redes de Computadores 1 INF 48 16 64 NE OBR
39 Sistemas Distribuídos INF 38 48 16 64 NE OBR
40 Sistemas Gerenciadores de Banco de Dados INF 6 64 0 64 NE OBR
41 Sistemas Operacionais INF 02, 05 64 0 64 NE OBR
42 Software Básico INF 2 0 64 64 NE OBR
43 Teoria dos Grafos INF 30 64 0 64 NE OBR
44 Aprendizagem de Máquina e Mineração de Dados INF 23 32 32 64 NE OPT
45 Armazém de Dados INF 40 48 16 64 NE OPT
46 Avaliação de Desempenho de Sistemas Computacionais INF 33 64 0 64 NE OPT
47 Banco de Dados Distribuídos INF 40 64 0 64 NE OPT
48 Computação Evolucionária INF 32 32 64 NE OPT
49 Computação Móvel e Ubíqua INF 34, 35 16 48 64 NE OPT
50 Computação Verde INF 38 32 32 64 NE OPT
51 Desenvolvimento de Sistemas para Web INF 34 32 32 64 NE OPT
52 Desenvolvimento de Software Dirigido por Modelos INF 02, 34 32 32 64 NE OPT
53 Empreendedorismo Digital INF 32 32 64 NE OPT
54 Engenharia de Sistemas INF 64 0 64 NE OPT
55 Engenharia de Software Baseada em Busca INF 17 32 32 64 NE OPT
56 Fundamentos da Programação Lógica e Funcional INF 27, 29 64 0 64 NE OPT
57 Integração de Dados INF 40 64 0 64 NE OPT
58 Introdução à Língua Brasileira de Sinais FL 64 0 64 NE OPT
59 Introdução à Teoria das Provas INF 29 64 0 64 NE OPT
60 Introdução à Teoria dos Tipos INF 29 64 0 64 NE OPT
61 Jogos Digitais INF 34 16 48 64 NE OPT
62 Meta-heurísticas INF 2 64 0 64 NE OPT
63 Metodologia e Experimentação em Engenharia de Software INF 64 0 64 NE OPT
64 Microcontroladores INF 10, 26 32 32 64 NE OPT
65 Processamento Digital de Imagens INF 01, 26 64 0 64 NE OPT
66 Recuperação de Informação INF 02, 33 64 0 64 NE OPT
67 Redes de Computadores 2 INF 38 32 32 64 NE OPT
68 Redes Neuronais Artificiais INF 23 32 32 64 NE OPT
69 Segurança e Auditoria de Sistemas INF 38 32 32 64 NE OPT
70 Sistemas Multiagentes INF 34 32 32 64 NE OPT
71 Tópicos Avançados em Redes Sem Fio INF 16 48 64 NE OPT
72 Tópicos em Computação INF 64 0 64 NE OPT
73 Tópicos em Teste de Software INF 48 16 64 NE OPT
74 Verificação Formal de Programas INF 29 32 32 64 NE OPT
75 Visão Robótica INF 01, 65 64 0 64 NE OPT
76 Visualização de Informações INF 3 32 32 64 NE OPT
77 Web Semântica INF 34 32 32 64 NE OPT
""".split('\n')[1:-1]

novos_requisitos = [ re.sub('[0-9]* [0-9]* [0-9]* [A-Z]* [A-Z]*$', '', line) for line in novos_requisitos ]

novos_requisitos = [ re.split('(^[0-9]* )', line)[1:] for line in novos_requisitos ]

novos_requisitos = [ [x[0]] + re.split(' [A-Z][A-Z][A-Z]? ', x[1]) for x in novos_requisitos ]

novos_requisitos = [ extend_list(x) for x in novos_requisitos ]

novos_requisitos = [
    {
        "key": int(x[0]),
        "name": x[1],
        "deps": list_to_int(x[2].split(','))
    } for x in novos_requisitos
]

novos_requisitos

materias_tabela = df[df['codigo_matriz_curricular']=='CICOMP-BI-3']['nome_componente'].drop_duplicates().values

grafo_requisitos = []
for x in materias_tabela:
    distances = []
    names = []
    for y in novos_requisitos:
        distances.append( levenshtein(x.lower(),y['name'].lower()) )
        names.append(y['name'])
    match = names[distances.index(min(distances))]
    for y in novos_requisitos:
        if y['name'] == match:
            y['name'] = x
            grafo_requisitos.append(y)

for item in grafo_requisitos:
    deps = item['deps']
    for idx,dep in enumerate(deps):
        for item2 in grafo_requisitos:
            if item2['key'] == dep:
                item['deps'][idx] = item2['name']

dependencias_grade_nova = []
for item in grafo_requisitos:
    for dep in item['deps']:
        row = {
            "materia": item['name'],
            "dependencia": dep
        }
        dependencias_grade_nova.append(row)
dependencias_grade_nova = pd.DataFrame(dependencias_grade_nova)

In [5]:
# Copiado de http://www.inf.ufg.br/sites/default/files/ProjetoPedagogico2008.pdf
requisitos = [
    {
    "materia": "Programação de Computadores 2",
    "dependencia": "Programação de Computadores 1"
},
    {
    "materia": "Estrutura de Dados I",
    "dependencia": "Programação de Computadores 1"
},
    {
    "materia": "Programação Orientada a Objetos",
    "dependencia": "Programação de Computadores 1"
},
    {
    "materia": "Linguagens de Programação",
    "dependencia": "Programação de Computadores 1"
},
    {
    "materia": "Estruturas de Dados II",
    "dependencia": "Estruturas de Dados I"
},
    {
    "materia": "Análise e Projeto de Algoritmos",
    "dependencia": "Estruturas de Dados I"
},
    {
    "materia": "Teoria da Computação",
    "dependencia": "Linguagens Formais e Autômatos"
},
    {
    "materia": "Compiladores",
    "dependencia": "Linguagens Formais e Autômatos"
},    
    {
    "materia": "Compiladores",
    "dependencia": "Estruturas de Dados II"
},
    {
    "materia": "Redes de Computadores 2",
    "dependencia": "Redes de Computadores 1"
},
    {
    "materia": "Sistemas Distribuídos",
    "dependencia": "Redes de Computadores 1"
},
    {
    "materia": "Inteligência Artificial",
    "dependencia": "Lógica Matemática"
},
    {
    "materia": "Banco de Dados",
    "dependencia": "Lógica Matemática"
},
    {
    "materia": "Sistemas Gerenciadores de BD",
    "dependencia": "Banco de Dados"
},
    {
    "materia": "Sistemas Operacionais 2",
    "dependencia": "Sistemas Operacionais 1"
},
    {
    "materia": "Cálculo 2",
    "dependencia": "Cálculo 1"
},
    {
    "materia": "Projeto Final de Curso 2",
    "dependencia": "Projeto Final de Curso 1"
}
]


materias_tabela = df[df['codigo_matriz_curricular']=='CICOMP-BI-2']['nome_componente'].drop_duplicates().values

for idx, req in enumerate(requisitos):
    for key, value in req.items():
        tmp = []
        for materia in materias_tabela:
            lev = levenshtein(value.lower(), materia.lower())
            if lev < 10:
                tmp.append( [ lev, materia ] )
        tmp = sorted(tmp, key=lambda x: x[0])
        try:
            new_value = tmp[0][1]
            requisitos[idx][key] = new_value
        except:
            pass
        
dependencias_grade_antiga = pd.DataFrame(requisitos)

In [6]:
materias_novas = df[df['codigo_matriz_curricular']=='CICOMP-BI-3']['nome_componente'].drop_duplicates().values
materias_antigas = df[df['codigo_matriz_curricular']=='CICOMP-BI-2']['nome_componente'].drop_duplicates().values

equivalencias = []
for mn in materias_novas:
    tmp = []
    for ma in materias_antigas:
        lev = levenshtein(mn, ma)
        if lev < 10:
            tmp.append( [levenshtein(mn, ma), ma] )
            tmp = sorted(tmp, key=lambda x: x[0])
    if not tmp:
        continue
    match = tmp[0][1]
    row = {
            "nova": mn,
            "antiga": match
        }
    equivalencias.append(row)
equivalencias = \
[{'nova': 'CÁLCULO 1A', 'antiga': 'CÁLCULO 1'},
 {'nova': 'CÁLCULO 2A', 'antiga': 'CÁLCULO 2'},
#  {'nova': 'FÍSICA III', 'antiga': 'TÓPICOS 1'},
 {'nova': 'ÁLGEBRA LINEAR', 'antiga': 'ÁLGEBRA LINEAR'},
 {'nova': 'PROBABILIDADE E ESTATÍSTICA A',
  'antiga': 'PROBABILIDADE E ESTATÍSTICA'},
#  {'nova': 'MATEMÁTICA DISCRETA', 'antiga': 'MATEMÁTICA DISCRETA'},
 {'nova': 'PROGRAMAÇÃO ORIENTADA A OBJETOS',
  'antiga': 'PROGRAMAÇÃO ORIENTADA A OBJETOS'},
 {'nova': 'ARQUITETURA DE COMPUTADORES',
  'antiga': 'ARQUITETURA DE COMPUTADORES'},
 {'nova': 'ANÁLISE E PROJETO DE ALGORITMOS',
  'antiga': 'ANÁLISE E PROJETO DE ALGORITMOS'},
 {'nova': 'BANCO DE DADOS', 'antiga': 'BANCO DE DADOS'},
 {'nova': 'COMPUTAÇÃO E SOCIEDADE', 'antiga': 'COMPUTADOR E SOCIEDADE'},
#  {'nova': 'FÍSICA I', 'antiga': 'TÓPICOS 1'},
 {'nova': 'LÓGICA MATEMÁTICA', 'antiga': 'LÓGICA MATEMÁTICA'},
 {'nova': 'ENGENHARIA DE SOFTWARE', 'antiga': 'ENGENHARIA DE SOFTWARE'},
 {'nova': 'INTRODUÇÃO À PROGRAMAÇÃO', 'antiga': 'INTRODUÇÃO À COMPUTAÇÃO'}]

manual = [
    {'nova': 'ALGORITMOS E ESTRUTURAS DE DADOS 1', 'antiga': 'ESTRUTURAS DE DADOS I'},
    {'nova': 'ALGORITMOS E ESTRUTURAS DE DADOS 2', 'antiga': 'ESTRUTURAS DE DADOS II'}]
equivalencias += manual
equivalencias = pd.DataFrame(equivalencias)

In [7]:
def aluno_no_fluxo(df, periodo, grade_nova=True):
    if grade_nova == True:
        idx = '3'
    else:
        idx = '2'
    tmp = df[df['codigo_matriz_curricular']== 'CICOMP-BI-' + idx]\
    .sort_values(by='semestre_previsto_componente')\
    [['nome_componente', 'semestre_previsto_componente']]
    tmp = tmp[tmp['semestre_previsto_componente'] <= periodo]\
    [['nome_componente']]\
    .drop_duplicates().values.flatten().tolist()
    return tmp
    
def materias_permitidas(materias_cursadas,
                        equivalencias,
                        dependencias_grade_antiga, 
                        dependencias_grade_nova,
                        grade_nova=True, 
                        grade_antiga=True):
    eq_dict = {}
    for item in equivalencias.values.tolist():
        eq_dict[item[0]] = item[1]
        eq_dict[item[1]] = item[0]
    tmp = []
    for materia in materias_cursadas:
        try:
            tmp.append(eq_dict[materia])
        except:
            continue
    materias_cursadas.extend(tmp)
    materias_cursadas = sorted(list(set(materias_cursadas)))
    todas_materias = df['nome_componente'].drop_duplicates().values.flatten().tolist()
    materias_novas = df[df['codigo_matriz_curricular']=='CICOMP-BI-3']['nome_componente'].drop_duplicates().values
    materias_antigas = df[df['codigo_matriz_curricular']=='CICOMP-BI-2']['nome_componente'].drop_duplicates().values
    
    permitidas = []
    if grade_antiga == True:
        for materia in materias_antigas:

            dep = dependencias_grade_antiga[dependencias_grade_antiga['materia']==materia]\
            ['dependencia'].drop_duplicates().values.flatten().tolist()
            truth_checks = [
                len(set(dep).intersection(set(materias_cursadas))) == len(set(dep)),
                materia not in materias_cursadas
            ]
            if all(truth_checks):
                permitidas.append(materia)
            
    if grade_nova == True:
        for materia in materias_novas:

            dep = dependencias_grade_nova[dependencias_grade_nova['materia']==materia]\
            ['dependencia'].drop_duplicates().values.flatten().tolist()
            truth_checks = [
                len(set(dep).intersection(set(materias_cursadas))) == len(set(dep)),
                materia not in materias_cursadas
            ]
            if all(truth_checks):
                permitidas.append(materia)
            
    return sorted(list(set(permitidas)))

universo = materias_permitidas(aluno_no_fluxo(df, 2, grade_nova=True),
                    equivalencias,
                    dependencias_grade_antiga, 
                    dependencias_grade_nova)

In [8]:
def otimizar_universo(df, universo, equivalencias):
    eq_dict = {}
    for item in equivalencias.values.tolist():
        eq_dict[item[0]] = item[1]
        eq_dict[item[1]] = item[0]
    notas = df[['nome_componente', 'nota_discente_componente']]
    notas = notas.groupby('nome_componente').mean().to_dict()['nota_discente_componente']
    universo_otimizado = []
    for item in universo:
        if item not in eq_dict.values():
            universo_otimizado.append(item)
            continue
        try:
            if eq_dict[item] in universo:
                n1 = notas[item]
                n2 = notas[eq_dict[item]]
                if n1 >= n2:
                    universo_otimizado.append(item)
                else:
                    universo_otimizado.append(eq_dict[item])
        except:
            continue
    return sorted(list(set(universo_otimizado)))

In [9]:
entradas = materias_permitidas(aluno_no_fluxo(df, 2, grade_nova=False),
                    equivalencias, 
                    dependencias_grade_antiga, 
                    dependencias_grade_nova,
                    grade_nova=False,
                   grade_antiga=True)

entradas = pd.DataFrame(entradas).rename(columns={
    0: "nome_componente"
})

carga = df[['nome_componente', 'ch_total_componente']].drop_duplicates()
notas = df[['nome_componente', 'nota_discente_componente']]\
.groupby('nome_componente')\
.mean()\
.reset_index()
atributos = pd.merge(carga, notas, how='inner', on='nome_componente')
entradas = pd.merge(atributos, entradas, how='inner', on='nome_componente')
entradas

,nome_componente,ch_total_componente,nota_discente_componente
0,ARQUITETURA DE COMPUTADORES,64,5.525963
1,ESTRUTURAS DE DADOS I,64,4.658454
2,PROGRAMAÇÃO ORIENTADA A OBJETOS,64,5.499179
3,TEORIA DOS GRAFOS,64,4.547887
4,BANCO DE DADOS,64,5.265583
5,LINGUAGENS DE PROGRAMAÇÃO,64,5.181967
6,PESQUISA OPERACIONAL,64,4.346325
7,ENGENHARIA DE SOFTWARE,64,6.132533
8,INTELIGÊNCIA ARTIFICIAL,64,6.466975
9,REDES DE COMPUTADORES 1,64,4.590956


In [17]:
def knapsack(C, wt, val): 
    K = [[0 for x in range(C+1)] for x in range(len(val)+1)] 

    for i in range(len(val)+1): 
        for w in range(C+1): 
            if i==0 or w==0: 
                K[i][w] = 0
            elif wt[i-1] <= w: 
                K[i][w] = max(val[i-1] + K[i-1][w-wt[i-1]],  K[i-1][w]) 
            else: 
                K[i][w] = K[i-1][w] 
  
    return K

def get_items(K, n, C, names):
    res = K[n][C]
    w = C
    for i in range(n, 0, -1): 
        if res <= 0: 
            break
        if res == K[i - 1][w]: 
            continue
        else:
            print(names[i - 1])
            res = res - val[i - 1] 
            w = w - wt[i - 1] 

C = 320
names = wt = entradas['nome_componente'].values.tolist()
wt = entradas['ch_total_componente'].values.tolist()
val = entradas['nota_discente_componente'].values.tolist()
get_items(knapsack(C, wt, val), len(val), C, names)

SEGURANÇA E AUDITORIA DE SISTEMAS
EMPREENDEDORISMO
COMPUTADOR E SOCIEDADE
DIREITO
TÓPICOS 1
PROJETO FINAL DE CURSO 1
PROJETO DE SOFTWARE
